In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.base import clone


with open('../../data/DatasetCleaned.csv', 'r') as f:
    DATA = pd.read_csv(f)


Y_DATA = np.log(DATA['SalePrice'].copy()).values
VAR_DEPENDENT = 'SalePrice_log'

X_DATA = DATA.copy().drop(columns=['SalePrice','Unnamed: 0'])


def PrepX(X:pd.DataFrame, scaler_type=PowerTransformer(method='yeo-johnson'), 
          only_dummies:bool=False, fit:bool=False):

    X_prep = pd.DataFrame(index=X.index)
    X_types = X.dtypes.to_dict()

    #Stores Scalers later in Dictionary (Column:Scaler)
    if fit: scalers = {}
    else: scalers = scaler_type

    new_cols = []

    #Iterates over X and applies transformations based on column type; Adds transformed columns 
    for col, t in X_types.items():

        #Dummy columns and Percentages that don't need to be scaled (computed in Preperation.ipynb)
        if t != 'object' and -1 <= X[col].min() and X[col].max() <= 1 and not only_dummies: 
            new_cols.append(X[col].copy().replace({-1:0})) 
            continue

        #Transforms Numeric values; either builds scalers or applies them
        if t in ['int', 'float'] and not only_dummies:
            nonapplicable = X[col] == -1

            new_col = X[col].copy()
            applic_values = new_col.loc[~nonapplicable].values.reshape(-1, 1)

            #If fit=True, construct a new scaler for the training data. Otherwise, use the SCALER dict to scale test data
            if fit: 
                scaler = clone(scaler_type)
                scaler.fit(applic_values)
                scalers[col] = scaler
                new_col.loc[~nonapplicable] = scaler.transform(applic_values).flatten()
            else:
                new_col.loc[~nonapplicable] = scalers[col].transform(applic_values).flatten()


            new_col.loc[nonapplicable] = 0

            new_cols.append(new_col)
            
            continue

        #Create dummies for categorical (str) columns; with Validation set this is done before splitting
        if t == 'object': 
            new_col = pd.get_dummies(X[col].copy(), prefix=col, drop_first=True, dtype=int)
            new_cols.append(new_col)
            continue

        #This is only relevant when creating only dummies for the validation Set to not loose columns
        if only_dummies and t != 'object':
            X_prep[col] = X[col].copy()
        else:
            print(f'Type \"{t}\" not in standard types!')

    if new_cols: X_prep = pd.concat([X_prep] + new_cols, axis=1)

    if fit: return X_prep, scalers
    else: return X_prep

#Split data into train and validation set. 
# #train_test_split is not because before splitting we need to ensure that all dummy variables are present to not have missing columns in either set
def Split(X:pd.DataFrame, Y:pd.DataFrame, TestSize:float, random_state:int=42):

    X = PrepX(X_DATA, only_dummies=True, fit=False)

    split_index = np.random.RandomState(random_state) \
                           .choice([True, False], size=len(Y), p=[TestSize,1-TestSize])
    
    X_test = X[split_index]
    X_train = X[~split_index]
    Y_test = Y[split_index]
    Y_train = Y[~split_index]

    return X_test, X_train, Y_test, Y_train


VALIDATION_SET = True
if VALIDATION_SET: 
    X_VAL, X_DATA, Y_VAL, Y_DATA = Split(X_DATA, Y_DATA, 0.05, 1)

X_DATA, SCALER = PrepX(X=X_DATA, fit=True)
VAR_NAMES = X_DATA.columns
X_DATA = X_DATA.values.reshape(-1,X_DATA.shape[1])

if VALIDATION_SET:
    X_VAL = PrepX(X=X_VAL, scaler_type=SCALER)
    X_VAL = X_VAL.values.reshape(-1, X_VAL.shape[1])

In [ ]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score

import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

def BootstrapLasso(Y, X, feature_names:list, alpha, n_bootstrap, display:bool=True):

    n_obs = len(Y)

    B = []
    FailedReg = 0
    for i in tqdm(range(0, n_bootstrap),
                  desc='Fitting Lasso-Model with Bootstrapped Samples', unit='sample'):
        sample_index = np.random.choice(range(0, n_obs), n_obs)

        X_sample = X[sample_index]
        Y_sample = Y[sample_index]

        lasso = LassoCV(alphas=alpha, cv=KFold(), max_iter=100000)
        lasso.fit(X_sample, Y_sample)        

        if lasso.alpha_ == min(alpha): 
            FailedReg += 1
            continue

        coefs = {}
        for i, var in enumerate(feature_names):
            coefs[var] = lasso.coef_[i]

        mse = np.mean((Y - lasso.predict(X))**2)

        B.append({
            'X_sample':X_sample,
            'Y_sample':Y_sample,
            'Sample_Key':sample_index,
            'LassoModel':lasso,
            'alpha':lasso.alpha_,
            'r_squared':lasso.score(X_sample, Y_sample),
            'r_squared_fullData':lasso.score(X, Y),
            'coefficients':coefs,
            'mse':mse,
            'N Nonzero Estimators':sum([1 for key, item in coefs.items() if item != 0])
        })


    print(f'{FailedReg} failed regularizations')
    return pd.DataFrame(B)

In [ ]:
import pickle

#with open('models/Bootstrapped_Lasso.sav', 'rb') as f: BootstrappedModels = pickle.load(f)
with open('models/Lasso.sav', 'rb') as f: LassoOrig, ScalerOrig = pickle.load(f)

#This takes about 30 Minutes to run
BootstrappedModels = BootstrapLasso(Y_DATA, X_DATA, VAR_NAMES, alpha=np.logspace(-4,-2,200), n_bootstrap=1000)
with open('models/Bootstrapping.sav', 'wb') as f: pickle.dump(BootstrappedModels, f)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def Coefficient_Boxplots(B_Coefficients:pd.Series, Lasso_Orig:LassoCV, var_names:list, cutoff:int=0.0):

    relevant_cols = []
    for i, var in enumerate(var_names):
        values = np.array([B[var] for B in B_Coefficients])
        if np.count_nonzero(values) / len(values) >= cutoff:
            relevant_cols.append({'var':var, 'coefs':values, 'index':i})
    
    boxplot_rows = int(np.ceil(len(relevant_cols) / 5))
    fig, axes = plt.subplots(boxplot_rows, 5, figsize=(15, 2*boxplot_rows))
    axes = axes.flatten()

    for ax, var in zip(axes, relevant_cols):
        ax.boxplot(var['coefs'], patch_artist=True)
        ax.axhline(y = 0, color='black', ls = '--')
        ax.axhline(y = Lasso_Orig.coef_[var['index']], color = 'red', ls=':')
        ax.set_title(var['var'], fontsize=8)

    plt.show()


In [ ]:
Coefficient_Boxplots(BootstrappedModels['coefficients'], LassoOrig, VAR_NAMES, 0.75)